In [41]:
from enum import Enum
class Lang(Enum):
    en = 1
    ru = 2

# choose language (affect movie dataset)
ENGINE = Lang.ru

In [42]:
# solve problem with mystem package in google colab
!if [[ "$OSTYPE" == "linux-gnu" ]]; then wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz && tar -xvf mystem-3.0-linux3.1-64bit.tar.gz && cp mystem /bin; fi

fish: Missing end to balance this if statement
if [[ "$OSTYPE" == "linux-gnu" ]]; then wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz && tar -xvf mystem-3.0-linux3.1-64bit.tar.gz && cp mystem /bin; fi
^



In [43]:
import spacy
import requests
import pandas as pd
from pymystem3 import Mystem

COLAB_ENV = "google.colab" in str(get_ipython())
ENV_PREFIX = "https://raw.githubusercontent.com/demimurg/fellini/master/datasets/" if COLAB_ENV else "./datasets/"


def get_stopwords():
    path = ENV_PREFIX + f"stopwords_{ENGINE.name}.txt"
    data = requests.get(path).text.split("\n") if COLAB_ENV else open(path)

    stopwords = set()
    for word in data:
        stopwords.add(word.rstrip())
    return stopwords


class EnLemmaTokenizer:
    def  __init__(self):
        self.nlp = spacy.load(ENGINE.name)
        self.stopwords = get_stopwords()
    def __call__(self, text):
        return [
            token.lemma_ for token in self.nlp(text)
            if token.lemma_ not in self.stopwords 
            and token.lemma_[0].isalpha()
        ]

class RuLemmaTokenizer:
    def  __init__(self):
        self.lemma = Mystem().lemmatize
        self.stopwords = get_stopwords()
    def __call__(self, text):
        return [
            token for token in self.lemma(text)
            if token not in self.stopwords and token[0].isalpha()
        ]

lemma_tokenizer = EnLemmaTokenizer() if ENGINE == Lang.en else RuLemmaTokenizer()

bunch = pd.read_csv(ENV_PREFIX + ("imdb250.csv" if ENGINE == Lang.en else "kp250.csv"))
bunch.fillna("", inplace=True)

In [44]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(max_features=50_000, tokenizer=lemma_tokenizer)
data = \
    bunch.Title + " " + bunch.Crew + " " + bunch.Plot + " " + \
    bunch.Meta + " " + bunch.Country + " " + bunch.Reviews

features = vec.fit_transform(data)

words = vec.get_feature_names()
print(f'Dictionary size = {len(words)}')

Dictionary size = 36669
CPU times: user 8.38 s, sys: 541 ms, total: 8.93 s
Wall time: 1min 12s


In [45]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=5, metric='cosine')
knn.fit(features)

NearestNeighbors(metric='cosine')

In [46]:
%%time

# write your search query
QUERY = "Где снимался Мэл Гибсон"
query_vector = vec.transform([QUERY])
print(f'Query = \'{QUERY}\'\nTokens =  {lemma_tokenizer(QUERY)}\n')

distances, neighbors = knn.kneighbors(query_vector, return_distance=True)
    
for dist, neighbor_idx in zip(distances[0], neighbors[0]):
    print(
        bunch.Title[neighbor_idx],
        f'Distance = {dist}  Neighbor idx = {neighbor_idx}',
        bunch.Plot[neighbor_idx][:200],
        bunch.Crew[neighbor_idx],
        "-"*200, sep="\n"
    )


Query = 'Где снимался Мэл Гибсон'
Tokens =  ['сниматься', 'мэл', 'гибсон']

Храброе сердце 
Distance = 0.6319537927903942  Neighbor idx = 64
Действие фильма начинается в 1280 году в Шотландии. Это история легендарного национального героя Уильяма Уоллеса, посвятившего себя борьбе с англичанами при короле Эдварде Длинноногом.Он рано лишился 
 Мэл Гибсон,  Рэндалл Уоллес, Мэл Гибсон, Софи Марсо, Патрик МакГуэн, Энгус Макфадьен, Брендан Глисон
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Апокалипсис 
Distance = 0.6925668731345805  Neighbor idx = 213
В фильме показана жизнь цивилизации майя до прихода испанских конкистадоров: жестокие войны с соседними племенами, человеческие жертвоприношения, загадочные мистические ритуалы. 

В один день рушится 
 Мэл Гибсон,  Мэл Гибсон,  Фарад Сафиниа, Руди Янгблад, Далия Эрнандес, Джонатан Брюэр, Мор